In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import root

 ### Merton Credit Model
 \begin{align}
 \ E = A_{MKT} N(d_1) - F e^{-rT} N(d_2) .... ⓐ\\ \\
 \ \sigma_E = \frac{A_{MKT}}{E} N(d_1) \sigma_A.... ⓑ \\
 \\
 \ Using \ Simultanous \ Equation \\
 \ Find \ Market \ Value \ of \ Asset \ and \ Volatility\ of \ Asset \ ( \ 
 A_{MKT} ,\ \sigma_E \ ) \\ \\ PD = 1 - N( d_2 ) \
 \end{align}

In [2]:
def Optimize_Merton(F,r,T,sigma_E,E_market_value) :
    E_market = E_market_value
    def EandsigmaE(x,F) :
        sig_A = abs(x[0])
        A = max(x[1], E_market_value)
        d1 = (np.log(A/F) + (r + 0.5 * sig_A**2)*T)/(sig_A * np.sqrt(T))
        d2 = d1 - sig_A * np.sqrt(T)
        Nd1 = norm.cdf(d1)
        Nd2 = norm.cdf(d2)
        E = max(A * Nd1 - F * np.exp(-r * T) * Nd2, 0.001)
        sig_E = A/E * Nd1 * sig_A
        return np.array([sig_E,E])
    def fun(x,F,ans) :
        return EandsigmaE(x,F) - ans

    x0 = np.array([sigma_E, int(F+E_market)])
    ret = root(fun, x0, args = (F, np.array([sigma_E,E_market]) ) )
    sigA, MV_Asset = ret.x[0], ret.x[1] ## Return sigA and Market Value of Asset
    return abs(sigA) , MV_Asset

def Merton_Debt_Value(F,r,T,sigma_E, E_market_value) :
    sig_A , A = Optimize_Merton(F, r, T, sigma_E, E_market_value)
    d1 = (np.log(A/F) + (r + 0.5 * sig_A**2)*T)/(sig_A * np.sqrt(T))
    d2 = d1 - sig_A * np.sqrt(T)
    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    E = A * Nd1 - F * np.exp(-r * T) * Nd2 
    D = A-E
    return D

def Merton_Default_Prob(F,r,T,sigma_E, E_market_value) :
    sig_A , A = Optimize_Merton(F, r, T, sigma_E, E_market_value)
    d1 = (np.log(A/F) + (r + 0.5 * sig_A**2)*T)/(sig_A * np.sqrt(T))
    d2 = d1 - sig_A * np.sqrt(T)
    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    PD = 1 - Nd2
    Annualized_PD = - 1/T * np.log(1-PD)
    return {'Annualized_PD':Annualized_PD, 'sig_A':sig_A, 'MV_Asset':A}


In [3]:
Merton_Default_Prob(F = 3000, r = 0.016, T = 10, sigma_E = 0.2, E_market_value = 5000)

{'Annualized_PD': 0.0008903807540535035,
 'sig_A': 0.13273566128967074,
 'MV_Asset': 7553.616715155387}